In [48]:
import os
import sys
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import json
import pathlib

from mpldatacursor import datacursor as cursor
from datetime import timedelta
from datetime import datetime

In [49]:
path  = "/mnt/bkt_prd_dsv_brain_therapy_raw/Brain Therapy MVP Content Pilot/"
directories = os.listdir(path)
directories = [d for d in directories if d.startswith('BT')]
directories

['BT-MVP-0001',
 'BT-MVP-0002',
 'BT-MVP-0003',
 'BT-MVP-0004',
 'BT-MVP-0005',
 'BT-MVP-0006',
 'BT-MVP-0007',
 'BT-MVP-0008',
 'BT-MVP-0009',
 'BT-MVP-0010',
 'BT-MVP-0011',
 'BT-MVP-0012',
 'BT-MVP-0013',
 'BT-MVP-0014',
 'BT-MVP-0015',
 'BT-MVP-0016',
 'BT-MVP-0017',
 'BT-MVP-0018',
 'BT-MVP-0019',
 'BT-MVP-0020',
 'BT-MVP-0021']

In [3]:
files = list([])
for directory in directories:
    f = os.listdir(path+"/"+directory)
    files.append([path+"/"+directory+"/"+x for x in f if x.endswith("total.txt")])
    

In [4]:
text_files = [x for x in files if len(x)==1]


In [5]:
data = pd.DataFrame([])
for i, participant_data in enumerate(text_files):
    for txt in participant_data:
        d = pd.read_csv(txt,skiprows = 15, usecols = [0,1,2,3,4], names = ['RSP','PPG','T','ECG','EDA'])
        d['participant_name'] = "participant_" + str(i+1)
        data = pd.concat([data, d])
data.set_index(['participant_name'], inplace = True)
data.head()

,RSP,PPG,T,ECG,EDA
participant_name,,,,,
participant_1,37272.000,-2.169495,-0.167542,31.185791,-0.325928
participant_1,37272.004,-2.178650,-0.166931,31.187622,-0.279541
participant_1,37272.008,-2.191772,-0.172729,31.186401,-0.226593
participant_1,37272.012,-2.201538,-0.177307,31.187622,-0.181427
participant_1,37272.016,-2.206421,-0.177612,31.186401,-0.151672


In [6]:
participants = data.index.get_level_values('participant_name').unique()

### JSON FILES OF EACH PARTICIPANT

In [28]:
def get_json(input_filename, output_filename):
    sk = re.compile(r'(?P<spaces>\s*)(?P<field>[a-zA-Z\-]+)(?P<colon> :)')
    out_filename = '/mnt/shared_drive/data/brain_therapy/json_output/'+output_filename
    with open(out_filename,'w') as out_file:
        with open(input_filename, 'r') as in_file:
            length = len(open(input_filename).readlines())
            for e,i in enumerate(in_file.readlines()):
                out_str = i.rstrip()
                if ('log' not in out_str) and (e!=length-2):
                    out_str = out_str.replace(']','}').replace('[','{')
                m = sk.search(out_str)
                if m:
                    out_file.write(sk.sub('\g<spaces>"\g<field>"\g<colon>',out_str)+'\n')
                else:
                    out_file.write(out_str+'\n')

In [37]:
path2  = "/mnt/bkt_prd_dsv_brain_therapy_raw/Brain Therapy MVP Content Pilot/"

json_files = list([])
for directory in directories:
    f = os.listdir(path2+directory)
    dimensions = ['2D','3D']
    for dimension in dimensions:
        d = [d_folder for d_folder in f if dimension in d_folder][0]
        d_files = os.listdir(path2+directory+"/"+d)
#         display(d_files)
        json_files.append([path2+directory+"/"+d+"/"+x for x in d_files if x.endswith(".json")])   

In [38]:
json_files = [j for j in json_files if len(j)>0]

In [39]:
json_files

[['/mnt/bkt_prd_dsv_brain_therapy_raw/Brain Therapy MVP Content Pilot/BT-MVP-0001/BT-MVP-0001-2D/BT-MVP-0001-1562790332-events.json'],
 ['/mnt/bkt_prd_dsv_brain_therapy_raw/Brain Therapy MVP Content Pilot/BT-MVP-0001/BT-MVP-0001-3D/BT-MVP-0001-3D-1562791350-events.json'],
 ['/mnt/bkt_prd_dsv_brain_therapy_raw/Brain Therapy MVP Content Pilot/BT-MVP-0002/BT-MVP-0002-2D/BT-MVP-0002-1562879278-events.json'],
 ['/mnt/bkt_prd_dsv_brain_therapy_raw/Brain Therapy MVP Content Pilot/BT-MVP-0002/BT-MVP-0002-3D/BT-MVP-0002-3D-1562878296-events.json'],
 ['/mnt/bkt_prd_dsv_brain_therapy_raw/Brain Therapy MVP Content Pilot/BT-MVP-0003/BT-MVP-0003-2D/BT-MVP-0003-2D-1563214950-events.json'],
 ['/mnt/bkt_prd_dsv_brain_therapy_raw/Brain Therapy MVP Content Pilot/BT-MVP-0003/BT-MVP-0003-3D/BT-MVP-0003-3D-1563215917-events.json'],
 ['/mnt/bkt_prd_dsv_brain_therapy_raw/Brain Therapy MVP Content Pilot/BT-MVP-0004/BT-MVP-0004-2D/BT-MVP-0004-2D-1563302076-events.json',
  '/mnt/bkt_prd_dsv_brain_therapy_raw/Bra

In [40]:
for json_ in json_files:
    output_filename = json_[0].split('/')[len(json_[0].split('/'))-1]
    get_json(json_[0], output_filename)

Stop here

In [42]:
events_df = pd.DataFrame([])
p = "/mnt/shared_drive/data/brain_therapy/json_output"
json_outputs = os.listdir(p)
for i, j in enumerate(sorted(json_outputs)):
    with open("{}/{}".format(p,j), 'r') as j_file:
        try:
            D = json.load(j_file)
            name = j_file.name.split("/")[6]
            num = name.split('-')[2]
        except:
            display(j_file)
            continue
        df = pd.DataFrame(D['log'])
        df['participant'] = "{}".format(num)
        df['type'] ="3D" if "3D" in name else "2D"
        events_df = pd.concat([events_df, df])
events_df.set_index(['participant','type'], inplace = True)

<_io.TextIOWrapper name='/mnt/shared_drive/data/brain_therapy/json_output/.BT-MVP-0012-2D-1563912525-events.json.swo' mode='r' encoding='UTF-8'>

<_io.TextIOWrapper name='/mnt/shared_drive/data/brain_therapy/json_output/.BT-MVP-0012-2D-1563912525-events.json.swp' mode='r' encoding='UTF-8'>

<_io.TextIOWrapper name='/mnt/shared_drive/data/brain_therapy/json_output/.BT-MVP-0021-2D-1565115325-events.json.swp' mode='r' encoding='UTF-8'>

<_io.TextIOWrapper name='/mnt/shared_drive/data/brain_therapy/json_output/.nfs0000000002c002b200000001' mode='r' encoding='UTF-8'>

/home/camar/.conda/envs/btsrd/lib/python3.7/site-packages/ipykernel/__main__.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [43]:
events_df['event'] = events_df['event'].fillna(events_df['eventName'])
events_df.drop(['eventName'], axis = 1, inplace = True)
events_df.set_index('event', append = True, inplace = True)
events_df.head()

details  \
participant type event                                                                 
0001        2D   ScriptStart       {'script-file': 'Test Scripts/mvp_balance_full...   
                 Script Log Event  {'context': 'StartFrontBuffer', 'time': '15627...   
                 RingHit                                                          {}   
                 RingHit                                                          {}   
                 RingHit                                                          {}   

                                    timestamp  
participant type event                         
0001        2D   ScriptStart       1562790332  
                 Script Log Event  1562790332  
                 RingHit           1562790335  
                 RingHit           1562790335  
                 RingHit           1562790335

In [44]:
log_events = pd.DataFrame([])
participants = events_df.index.get_level_values('participant').unique()
for participant in participants:
    data = events_df.loc(axis = 0 )[participant,:,'Script Log Event']
    details = data['details'].values
    context = [[x['context'], x['time']] for x in details]
    df = pd.DataFrame(context, columns = ['context','time'], index = data.index)
    log_events = pd.concat([log_events, df])
log_events.reset_index(["event"], drop = True, inplace = True)

In [45]:
log_events['time'] = pd.to_datetime(log_events['time'], unit = 's')
log_events['time'] = log_events['time']-timedelta(hours = 5)


In [47]:
joblib.dump(log_events, "/mnt/shared_drive/data/brain_therapy/log_events.pkl")

['/mnt/shared_drive/data/brain_therapy/log_events.pkl']

### 2D and 3D Skin Temperature and PPG Signals

In [16]:
signals = joblib.load("/mnt/shared_drive/data/brain_therapy/signals_dict.pkl")

In [17]:
def plot_signals(data):
    col = data.columns.values[0]
    dimensions = data.index.get_level_values('type').unique().values
    fig, axes = plt.subplots(2, 2, figsize = (15, 7))
    ax = axes.flatten()
    i = 0
    for dimension in dimensions:
        ax[i].plot(data.loc(axis = 0)[:,dimension][col].values)
        ax[i].set_title("{} {} data of {}".format(dimension,col, data.index.get_level_values('participant_name').unique().values[0]))
        
        ax[i+2].plot(data.loc(axis = 0)[:,dimension][col].values)
        ax[i+2].set_xlim(1000,5000)
        ax[i+2].set_title("{} {} ZOOMED data of {}".format(dimension,col, data.index.get_level_values('participant_name').unique().values[0]))
        i+=1

In [18]:
signal_df = pd.DataFrame([])
for participant in signals.keys():
    data = signals[participant]
    data['participant'] = participant
    signal_df = pd.concat([signal_df, data])
signal_df.set_index(['participant','Datetime'], inplace = True)

In [19]:
signal_df.head()

RSP       PPG     1-SKTA       ECG  \
participant Datetime                                                           
0001        2019-07-10 15:22:31.000 -1.263123  0.003052  12.941162 -0.234833   
            2019-07-10 15:22:31.004 -1.261902  0.003662  12.945435 -0.226593   
            2019-07-10 15:22:31.008 -1.264648  0.002136  12.946045 -0.215607   
            2019-07-10 15:22:31.012 -1.261902  0.003662  12.945435 -0.206604   
            2019-07-10 15:22:31.016 -1.262207  0.003357  12.946045 -0.198364   

                                          EDA  
participant Datetime                           
0001        2019-07-10 15:22:31.000  0.041124  
            2019-07-10 15:22:31.004  0.042649  
            2019-07-10 15:22:31.008  0.044175  
            2019-07-10 15:22:31.012  0.039598  
            2019-07-10 15:22:31.016  0.042649

In [20]:
signal_df.sort_index(sort_remaining = True, inplace = True)
participants = signal_df.index.get_level_values('participant').unique()
for participant in participants:
    sensors = signal_df.columns.values
#     fig, axes = plt.subplots(5,1, figsize = (18,10))
#     ax = axes.flatten()
    for i, sensor in enumerate(sensors):
        try:
            d = signal_df.loc(axis = 0)[participant,:][[sensor]]
#             ax[i].plot(d[sensor].values)
#             ax[i].set_title("{} of participant-{}".format(sensor,participant))
        except:
            print("{} has no {}".format(participant,sensor)) 

In [21]:
files2 = list([])
path2 = "/mnt/shared_drive/data/brain_therapy/Brain Therapy MVP Content Pilot/Timestamp-Export/MVP"
for directory in directories:
    f = os.listdir(path2+"/"+directory)
    files2.append([path2+"/"+directory+"/"+x for x in f if x.endswith("csv.txt")])
    

In [22]:
files2 = [file for file in files2 if len(file)>0]

In [23]:
signal_df2 = pd.DataFrame([])
for file in files2:
    temp = pd.DataFrame([])
    for data in file:
        name = data.split("/")[8].split("-")[2]
#         display(name)
        d = pd.read_csv(data, skiprows = 15, usecols = [0,1,2,3,4], names = ['RSP','PPG','T','ECG','EDA'])
        d['type'] = "3D" if "3D" in data else "2D"
        d['participant']=name
        temp = pd.concat([temp,d])
    #Check if 2D or 3D is longer
    dd = temp[temp['type']=='2D']
    ddd = temp[temp['type']=='3D']
    if len(dd) > len(ddd):
        signal_df2 = pd.concat([signal_df2,dd])
    else:
        signal_df2 = pd.concat([signal_df2, ddd])
        

In [24]:
signal_df2.set_index(['participant','type'], inplace = True)

In [25]:
participants = signal_df2.index.get_level_values('participant').unique().values
#Eliminating participant 0004, 0005,0009 because they don't have a *total.txt file
participants = [p for p in participants if p not in ['0004','0005','0009']]
new_signal_df = pd.DataFrame([])
for participant in participants:
    datetimes = signal_df.loc(axis = 0)[participant].index.get_level_values('Datetime').values
    d = signal_df2.loc(axis = 0)[participant,:]
    d['Datetime'] = datetimes
    new_signal_df = pd.concat([new_signal_df,d])
new_signal_df.rename(columns = {'T':'1-SKTA'}, inplace = True)

/home/camar/.conda/envs/btsrd/lib/python3.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [26]:
new_signal_df.set_index("Datetime", append = True, inplace = True)

In [27]:
display(new_signal_df.head())
display(signal_df.head())

RSP       PPG     1-SKTA  \
participant type Datetime                                                 
0017        3D   2019-07-30 10:21:12.000 -2.169495 -0.167542  31.185791   
                 2019-07-30 10:21:12.004 -2.178650 -0.166931  31.187622   
                 2019-07-30 10:21:12.008 -2.191772 -0.172729  31.186401   
                 2019-07-30 10:21:12.012 -2.201538 -0.177307  31.187622   
                 2019-07-30 10:21:12.016 -2.206421 -0.177612  31.186401   

                                               ECG       EDA  
participant type Datetime                                     
0017        3D   2019-07-30 10:21:12.000 -0.325928  8.377075  
                 2019-07-30 10:21:12.004 -0.279541  8.375550  
                 2019-07-30 10:21:12.008 -0.226593  8.372498  
                 2019-07-30 10:21:12.012 -0.181427  8.375550  
                 2019-07-30 10:21:12.016 -0.151672  8.375550

RSP       PPG     1-SKTA       ECG  \
participant Datetime                                                           
0001        2019-07-10 15:22:31.000 -1.263123  0.003052  12.941162 -0.234833   
            2019-07-10 15:22:31.004 -1.261902  0.003662  12.945435 -0.226593   
            2019-07-10 15:22:31.008 -1.264648  0.002136  12.946045 -0.215607   
            2019-07-10 15:22:31.012 -1.261902  0.003662  12.945435 -0.206604   
            2019-07-10 15:22:31.016 -1.262207  0.003357  12.946045 -0.198364   

                                          EDA  
participant Datetime                           
0001        2019-07-10 15:22:31.000  0.041124  
            2019-07-10 15:22:31.004  0.042649  
            2019-07-10 15:22:31.008  0.044175  
            2019-07-10 15:22:31.012  0.039598  
            2019-07-10 15:22:31.016  0.042649

In [47]:
import math
mvp_offset = pd.read_csv('MVP-Offsets_v2.csv')
mvp_offset['participant'] =mvp_offset['MVP ID'].astype(int)
mvp_offset['participant'] = mvp_offset['participant'].apply(lambda x: str(int(x)).zfill(4))
mvp_offset.set_index(['participant',"MVP ID"], inplace = True)
mvp_offset.drop("Insert Time", axis = 1, inplace = True)
mvp_offset.dropna(inplace = True)

In [48]:
mvp_offset['adjusted_insert_time'] = pd.to_datetime(mvp_offset['Adjusted Insert Time'], unit = 'ms')
mvp_offset['adjusted_insert_time'] = mvp_offset['adjusted_insert_time'].apply(lambda x : x.time())
mvp_offset.drop("Adjusted Insert Time", axis = 1, inplace = True)
mvp_offset.head()

Offset Time adjusted_insert_time
participant MVP ID                                 
0001        1.0       15:44:09      15:38:38.644000
0002        2.0       16:08:34      16:04:34.936000
0003        3.1       13:22:35      13:20:08.900000
            3.2       13:39:35      13:32:50.656000
0006        6.1       10:15:29      10:14:05.472000

In [30]:
temp = new_signal_df.reset_index('type', drop = True)
from matplotlib.pyplot import cm
for participant in sorted(participants):
    new = temp.loc(axis = 0)[participant,:]
    sig = signal_df.loc(axis = 0)[participant,:]
    fig, axes = plt.subplots(5,1, figsize = (18,25))
    ax = axes.flatten()
    plt.subplots_adjust(hspace = 0.5)
    insert = mvp_offset.loc(axis = 0)[participant,:]['adjusted_insert_time'].values
    offset = mvp_offset.loc(axis = 0)[participant,:]['Offset Time'].values
    colors = iter(cm.tab20(np.linspace(0,1, 25)))
    for z,sensor in enumerate(sensors):
        plot = ax[z].plot((new.index.get_level_values('Datetime')).time,new[sensor].values, '-o',zorder = 0, color = 'darkgrey')
        for ins,off in zip(insert,offset):
            i = datetime.strptime(ins, "%H:%M:%S").time()
            o = datetime.strptime(off,"%H:%M:%S").time()
            mvp = mvp_offset.loc(axis = 0)[participant,:][mvp_offset['Insert Time']==ins]
            mvp_id = mvp.index.get_level_values('MVP ID').unique().values[0]
            
            ax[z].axvspan(i,o, color = next(colors), zorder = 1, alpha = 0.4, label = mvp_id)
#             llim, ulim = ax[z].get_ylim()
#             ax[z].annotate("{}".format(mvp_id), (i,ulim), rotation = 90)
        ax[z].set_title("{} of Participant-{}".format(sensor, participant))
        ax[z].legend(loc ='upper right')
plt.show()

NameError: name 'new_signal_df' is not defined

In [49]:
# joblib.dump(mvp_offset, "/mnt/shared_drive/data/brain_therapy/mvp_offset.pkl")

['/mnt/shared_drive/data/brain_therapy/mvp_offset.pkl']